In [103]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('second-query').getOrCreate()


In [104]:
user_data_columns = ['user-id','item-id','rating','timestamp']
user_data_df = spark.read.csv('u.data',sep="\t")

from functools import reduce
oldColumns = user_data_df.columns

user_data_df = reduce(lambda df,ids: df.withColumnRenamed(oldColumns[ids],user_data_columns[ids]),range(len(oldColumns)),user_data_df)

In [105]:
items_df = spark.read.csv('u.item',sep='|')

item_old_columns = items_df.columns
item_new_columns = ['item-id','movie-title','release-date','video-release-data','IMDB-url','unknown','action','adventure','animation',"children's",'comedy','crime','documentary','drama','fantasy','film-noir','horror','musical','mystery','romance','sci-fi','thriller','war','western']


items_df = reduce(lambda df,ids: df.withColumnRenamed(item_old_columns[ids],item_new_columns[ids]),range(len(item_old_columns)),items_df)

In [106]:
rating_df = items_df.join(user_data_df,on='item-id')

In [107]:
rating_df.show()

+-------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------+------+---------+
|item-id|         movie-title|release-date|video-release-data|            IMDB-url|unknown|action|adventure|animation|children's|comedy|crime|documentary|drama|fantasy|film-noir|horror|musical|mystery|romance|sci-fi|thriller|war|western|user-id|rating|timestamp|
+-------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+----------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+-------+------+---------+
|    242|        Kolya (1996)| 24-Jan-1997|              null|http://us.imdb.co...|      0|     0|        0|        0|         0|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|  

In [108]:
from pyspark.sql.functions import count, avg, desc

rating_count_df = rating_df.groupBy('item-id','movie-title').agg(count("*").alias('rating_count'))
rating_count_df.show(truncate=False)

+-------+----------------------------------+------------+
|item-id|movie-title                       |rating_count|
+-------+----------------------------------+------------+
|1016   |Con Air (1997)                    |137         |
|870    |Touch (1997)                      |9           |
|689    |Jackal, The (1997)                |87          |
|261    |Air Bud (1997)                    |43          |
|388    |Beverly Hills Cop III (1994)      |28          |
|1537   |Cosi (1996)                       |4           |
|1590   |To Have, or Not (1995)            |2           |
|677    |Fire on the Mountain (1996)       |1           |
|1365   |Johnny 100 Pesos (1993)           |2           |
|1618   |King of New York (1990)           |1           |
|508    |People vs. Larry Flynt, The (1996)|215         |
|182    |GoodFellas (1990)                 |226         |
|669    |Body Parts (1991)                 |13          |
|1184   |Endless Summer 2, The (1994)      |10          |
|443    |Birds

In [109]:
df_filtered = rating_count_df.filter("rating_count>=35")
df_filtered.show()

+-------+--------------------+------------+
|item-id|         movie-title|rating_count|
+-------+--------------------+------------+
|   1016|      Con Air (1997)|         137|
|    689|  Jackal, The (1997)|          87|
|    261|      Air Bud (1997)|          43|
|    508|People vs. Larry ...|         215|
|    182|   GoodFellas (1990)|         226|
|    443|   Birds, The (1963)|         162|
|    363| Sudden Death (1995)|          47|
|    754|   Red Corner (1997)|          57|
|    434|Forbidden Planet ...|          67|
|    794|It Could Happen t...|          46|
|    106|   Diabolique (1996)|          71|
|    380|Star Trek: Genera...|         116|
|   1134|Get on the Bus (1...|          38|
|    716|Home for the Holi...|          58|
|     60|Three Colors: Blu...|          64|
|    654|    Chinatown (1974)|         147|
|    276|Leaving Las Vegas...|         298|
|    245|Devil's Own, The ...|         240|
|    585|   Son in Law (1993)|          39|
|     14|  Postino, Il (1994)|  

In [110]:
rating_avg_df = rating_df.groupBy('item-id','movie-title').agg({'rating':'mean'}).alias("rating-average")
rating_avg_df = rating_avg_df.drop('movie-title')
rating_avg_df.show(truncate=False)

+-------+------------------+
|item-id|avg(rating)       |
+-------+------------------+
|1016   |3.45985401459854  |
|870    |2.7777777777777777|
|689    |3.1494252873563218|
|261    |2.558139534883721 |
|388    |2.392857142857143 |
|1537   |4.0               |
|1590   |2.0               |
|677    |3.0               |
|1365   |1.5               |
|1618   |1.0               |
|508    |3.5953488372093023|
|182    |3.9513274336283186|
|669    |1.6153846153846154|
|1184   |2.5               |
|443    |3.808641975308642 |
|363    |2.723404255319149 |
|754    |3.3859649122807016|
|1169   |4.1               |
|434    |3.8358208955223883|
|794    |3.652173913043478 |
+-------+------------------+
only showing top 20 rows



In [111]:
final_join_df = df_filtered.join(rating_avg_df,on='item-id')

final_join_df.show()

+-------+--------------------+------------+------------------+
|item-id|         movie-title|rating_count|       avg(rating)|
+-------+--------------------+------------+------------------+
|   1016|      Con Air (1997)|         137|  3.45985401459854|
|    689|  Jackal, The (1997)|          87|3.1494252873563218|
|    261|      Air Bud (1997)|          43| 2.558139534883721|
|    508|People vs. Larry ...|         215|3.5953488372093023|
|    182|   GoodFellas (1990)|         226|3.9513274336283186|
|    443|   Birds, The (1963)|         162| 3.808641975308642|
|    363| Sudden Death (1995)|          47| 2.723404255319149|
|    754|   Red Corner (1997)|          57|3.3859649122807016|
|    434|Forbidden Planet ...|          67|3.8358208955223883|
|    794|It Could Happen t...|          46| 3.652173913043478|
|    106|   Diabolique (1996)|          71| 2.887323943661972|
|    380|Star Trek: Genera...|         116| 3.336206896551724|
|   1134|Get on the Bus (1...|          38|3.3157894736

In [112]:
sorted_df = final_join_df.sort(desc('avg(rating)'))

In [113]:
top_20_names_df = sorted_df.select('movie-title').limit(20)


In [114]:
top_20_names_df.show(truncate=False)

+------------------------------------------------------+
|movie-title                                           |
+------------------------------------------------------+
|Close Shave, A (1995)                                 |
|Schindler's List (1993)                               |
|Wrong Trousers, The (1993)                            |
|Casablanca (1942)                                     |
|Wallace & Gromit: The Best of Aardman Animation (1996)|
|Shawshank Redemption, The (1994)                      |
|Rear Window (1954)                                    |
|Usual Suspects, The (1995)                            |
|Star Wars (1977)                                      |
|12 Angry Men (1957)                                   |
|Third Man, The (1949)                                 |
|Citizen Kane (1941)                                   |
|Some Folks Call It a Sling Blade (1993)               |
|To Kill a Mockingbird (1962)                          |
|One Flew Over the Cuckoo's Nes